In [9]:
import requests
import pandas as pd
import time
import math

API_KEYS = [
    "468fd187-9460-42b3-adcb-934315fb3c60", 
    "f68ae405-a48a-432d-a4f8-1847ee3afd3c",
    "69969001-6dea-4fa0-8490-533cff7aabcf",
    "66715fc5-9def-486c-8cb7-c6c943833fd1",
    "4614076f-20c1-4632-941c-8a47dec5d386",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "06b3535f-186f-4cb7-be6d-14ae08e4b48b",
    "1f08fade-c545-41ef-b648-4610002ac2a5",
    "5e233917-0ea1-4f69-93a4-d4b2eb1b997f",
    "c2a0bca2-4dc2-4c37-bad6-7dad1ba17975",
    "0ff9b2cf-abfb-4efe-a148-e30485cfe3cb"
]

BATCH_SIZE = 150   # kamu bisa ubah sesuai kebutuhan


def check_number_status(phone_number, api_key, timeout=60):
    url = "https://api.starsender.online/api/check-number"
    headers = {
        "Content-Type": "application/json",
        "Authorization": api_key
    }
    payload = {"number": str(phone_number)}

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        r = response.json()
        return r.get("data", {}).get("message", "No message")
    except requests.exceptions.Timeout:
        return "Timeout"
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


def process_dataframe(df):
    total_rows = len(df)
    total_keys = len(API_KEYS)

    # hitung berapa batch
    total_batches = math.ceil(total_rows / BATCH_SIZE)
    print(f"Total rows: {total_rows}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Total batches: {total_batches}\n")

    for batch_index in range(total_batches):
        start_row = batch_index * BATCH_SIZE
        end_row = min(start_row + BATCH_SIZE, total_rows)

        print(f"\n=== Processing Batch {batch_index+1}/{total_batches} | Rows {start_row}–{end_row-1} ===\n")

        # copy batch untuk disimpan nanti
        batch_df = df.iloc[start_row:end_row].copy()

        for i in range(start_row, end_row):
            # putar API key berdasarkan row global index
            key_index = i % total_keys
            current_api_key = API_KEYS[key_index]

            phone_number = df.loc[i, "Whatsapp"]
            result = check_number_status(phone_number, current_api_key)

            batch_df.loc[i, "validity"] = result

            print(f"Row {i+1}/{total_rows} | API {key_index+1}/{total_keys} | {phone_number} → {result}")

            time.sleep(5)

        # simpan batch ke file
        filename = f"batch_{batch_index+1}.xlsx"
        batch_df.to_excel(filename, index=False)
        print(f"\n💾 Saved: {filename}")

    print("\n🎉 All batches processed successfully!")


In [2]:
# load data
database = pd.read_excel("cek_ricek.xlsx")

# load data validity (consist whatsapp number and the status registered or not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")

In [3]:
no_crm = database[database['CRM'].isna()]
uncheck = no_crm[(no_crm['validity'].isna()) & (no_crm['Source'] == 'Web Ads')].reset_index(drop=True)

In [6]:
uncheck['Whatsapp'] = uncheck['Whatsapp'].astype(int)

In [10]:
process_dataframe(uncheck)

Total rows: 147
Batch size: 150
Total batches: 1


=== Processing Batch 1/1 | Rows 0–146 ===



Row 1/147 | API 1/11 | 6285659253233 → Number not registered
Row 2/147 | API 2/11 | 6289682898392 → Number not registered
Row 3/147 | API 3/11 | 6287731953021 → Number registered
Row 4/147 | API 4/11 | 6282289047521 → Number registered
Row 5/147 | API 5/11 | 6282210739298 → Number not registered
Row 6/147 | API 6/11 | 6285315870662 → Number not registered
Row 7/147 | API 7/11 | 6285296317417 → Number not registered
Row 8/147 | API 8/11 | 6285720280430 → Number not registered
Row 9/147 | API 9/11 | 6285718055438 → Number registered
Row 10/147 | API 10/11 | 628700111759 → Number not registered
Row 11/147 | API 11/11 | 62815527555 → Number not registered
Row 12/147 | API 1/11 | 628771108946 → Number not registered
Row 13/147 | API 2/11 | 6289668171601 → Number registered
Row 14/147 | API 3/11 | 628547268921 → Number not registered
Row 15/147 | API 4/11 | 6281232729000 → Number registered
Row 16/147 | API 5/11 | 62885659253233 → Number not registered
Row 17/147 | API 6/11 | 628154540840 → 

In [11]:
# process_dataframe(uncheck)
# val = pd.read_excel("final_result.xlsx")
val_1 = pd.read_excel("batch_1.xlsx")
val_2 = pd.read_excel("batch_2.xlsx")
# val_3 = pd.read_excel("batch_3.xlsx")

In [12]:
validity = pd.concat([val_1, val_2])  # , val_2, val_3])
validity

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,...,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,Status,validity
0,46530,6282128802917,Zaenuddin,May,2025,NaN,Web Ads,600712,2,PALESTINA,...,05-06,300356,Thu,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
1,48027,628126442337,adelinda pulungan,June,2025,NaN,Web Ads,201297,2,PALESTINA,...,06-07,100648,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
2,52277,6281347808814,Hamba allah,September,2025,NaN,Web Ads,200296,2,PALESTINA,...,19-20,100148,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
3,54955,628121918596,Joe,October,2025,NaN,Web Ads,200789,2,PALESTINA,...,08-09,100394,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
4,55045,628345400400,Andi sudirman,October,2025,NaN,Web Ads,351135,2,PALESTINA,...,14-15,175568,Mon,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number not registered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,45543,6281289426669,Firdaus,November,2025,NaN,Web Ads,53000,1,PALESTINA,...,07-08,53000,Mon,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Pernah Sukses,Number registered
133,45544,6281229252349,Sugito,November,2025,NaN,Web Ads,100000,1,SUDAN,...,22-23,100000,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Error: 400 Client Error: Bad Request for url: ...
134,45545,6285211979798,Jailani,November,2025,NaN,Web Ads,100714,1,SUDAN,...,07-08,100714,Mon,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered
135,45547,6285795927968,royani ahmad,November,2025,NaN,Web Ads,100780,1,PALESTINA,...,06-07,100780,Mon,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Pernah Sukses,Number registered


In [13]:
validity_update = pd.concat([data_with_validity, validity])
validity_update = validity_update.drop_duplicates(subset=['Whatsapp'], keep='last').reset_index(drop=True)

In [14]:
validity_update['validity'].value_counts(dropna=False)

validity
NaN                                                                                                                                                21533
Number registered                                                                                                                                  19535
Number not registered                                                                                                                               2703
Error: 400 Client Error: Bad Request for url: https://api.starsender.online/api/check-number                                                          24
Error: 504 Server Error: Gateway Time-out for url: https://api.starsender.online/api/check-number                                                     14
Error: 502 Server Error: Bad Gateway for url: https://api.starsender.online/api/check-number                                                           4
Error: ('Connection aborted.', ConnectionResetError(10054, 'An existing c

In [15]:
validity_update = validity_update[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'CRM', 'Source',
       'Total', 'Frekuensi', 'klasifikasi_program', 'Preferensi', 'Label_Jam',
       'Rata - rata', 'Day_Mode', 'Date_Category', 'Tahun_Pertama', 'Kategori',
       'Badge', 'Avg Kategori', 'validity']]

In [16]:
validity_update.to_excel("checkpoint_files/data_with_number_validity_updated.xlsx")